In [1]:
%load_ext autoreload
%autoreload 2

from rtfs.chunk_resolution.chunk_graph import ChunkGraph
from src.config import GRAPH_ROOT
import json

repo_path = r"C:\Users\jpeng\Documents\projects\codesearch-backend\src\cluster\repos\aider"
graph_path = GRAPH_ROOT / "Aider-AI_aider.git"

graph_json = json.loads(open(graph_path, "r").read())
cg = ChunkGraph.from_json(graph_path, graph_json)

In [8]:
clusters = cg.get_clusters(return_content=True)
one_lvl = [c for cluster in clusters for c in cluster.children]
for c in one_lvl:
    print(c.to_str())
    # print(c.title)
    # print(c.summary)


Cluster 2: Code Modification and Editing Framework
Summary: This code provides a framework for modifying and editing code using various coder classes that implement search/replace block strategies. It includes different coder classes for handling specific types of code modifications, such as fenced blocks and whole file edits, and utilizes prompts to guide the editing process.
Chunks (7):
  Chunk: #16.26
  Chunk: #65.35
  Chunk: #80.9
  Chunk: #81.95
  Chunk: #85.54
  Chunk: #88.7
  Chunk: #89.16

Cluster 1: Architectural Code Editing and Prompt Management
Summary: This code feature provides a framework for managing and executing code edits based on architectural prompts. It includes classes for handling different types of code prompts and edits, allowing for structured and guided modifications to code files.
Chunks (11):
  Chunk: #17.44
  Chunk: #18.40
  Chunk: #19.9
  Chunk: #20.33
  Chunk: #63.52
  Chunk: #84.27
  Chunk: #86.87
  Chunk: #87.53
  Chunk: #111.27
  Chunk: #125.36
  Chu

In [7]:
code_mod = one_lvl[0]
print(code_mod.to_str(return_content=True))

Cluster 2: Code Modification and Editing Framework
Summary: This code provides a framework for modifying and editing code using various coder classes that implement search/replace block strategies. It includes different coder classes for handling specific types of code modifications, such as fenced blocks and whole file edits, and utilizes prompts to guide the editing process.
Chunks (7):
  Chunk: #16.26
  from .architect_coder import ArchitectCoder
  from .ask_coder import AskCoder
  from .base_coder import Coder
  from .editblock_coder import EditBlockCoder
  from .editblock_fenced_coder import EditBlockFencedCoder
  from .editor_editblock_coder import EditorEditBlockCoder
  from .editor_whole_coder import EditorWholeFileCoder
  from .help_coder import HelpCoder
  from .udiff_coder import UnifiedDiffCoder
  from .wholefile_coder import WholeFileCoder
  
  # from .single_wholefile_func_coder import SingleWholeFileFunctionCoder
  
  __all__ = [
      HelpCoder,
      AskCoder,
      Co

In [2]:
cg.build_cluster_edges()

Adding edge:  Coder 2 5
Adding edge:  Coder 2 5
Adding edge:  CoderPrompts 2 1
Adding edge:  Coder 1 5
Adding edge:  Coder 1 5
Adding edge:  InputOutput 4 9
Adding edge:  Coder 4 5
Adding edge:  format_content 7 6
Adding edge:  dump 7 3
Adding edge:  Coder 8 5
Adding edge:  CoderPrompts 8 1
Adding edge:  Coder 13 5
Adding edge:  CoderPrompts 13 1
Adding edge:  Coder 14 5
Adding edge:  CoderPrompts 14 1
Adding edge:  Coder 15 5
Adding edge:  dump 9 3
Adding edge:  Coder 9 5


In [5]:
print(cg.get_longest_path())

['Code Modification and Editing Framework', 'Architectural Code Editing and Prompt Management', 'Code Editing and Management System', 'Code Editing and Management']
['Codebase Management and Interaction System', 'Aider Codebase Management and Testing Framework', 'Code Editing and Diff Management System', 'Code Editing and Management']
['Codebase Management and Interaction System', 'Aider Codebase Management and Testing Framework', 'Code Editing and Management System', 'Code Editing and Management']
['Whole File Code Editing System', 'Architectural Code Editing and Prompt Management', 'Code Editing and Management System', 'Code Editing and Management']
['Interactive Help and Documentation Feature for Aider', 'Architectural Code Editing and Prompt Management', 'Code Editing and Management System', 'Code Editing and Management']
['Unified Diff Code Modification System', 'Architectural Code Editing and Prompt Management', 'Code Editing and Management System', 'Code Editing and Management']